# Getting Started

This tutorial will help you install the ESGF End-user API and demonstrate some of the capabilites using ESGF's WPS server on [aims2](https://aims2.llnl.gov).

## Step 1

Install miniconda for your OS.

* Miniconda for [MacOS](https://conda.io/docs/user-guide/install/macos.html) follow those instructions.

* Miniconda for [Linux](https://conda.io/docs/user-guide/install/linux.html) follow those instructions.

## Step 2

Install the required packages from conda.

```
conda install -c conda-forge -c cdat esgf-compute-api
```

To add the other packages we'll be using in this guide run the following:

```
conda install -c conda-forge -c cdat cdms2 vcs
```

If you're running a headless server you may need to install the mesalib version of vcs:

```
conda install -c conda-forge -c cdat mesalib cdms2 vcs
```

## Step 3

Next you'll want to create an account on [aims2](https://aims2.llnl.gov/wps/home) and retrieve an API key, giving you access
to LLNL's ESGF Compute Node. Follow the instructions on the landing page.

## Step 4

Now we'll run through some basics of using the ESGF Compute End-user API.

## Setup your connection the compute server

In [ ]:
api_key = 'TOKEN'

import cwt

wps = cwt.WPSClient('https://aims2.llnl.gov/wps/', api_key=api_key, verify=False)

## List the available processes
This will print out a list of the available processes.

In [ ]:
for p in wps.processes():
    print p.identifier

## Search for a specific process

This will find all processes matching the regular expression (https://docs.python.org/2/library/re.html).

In [ ]:
for p in wps.processes('.*\.subset'):
    print p.identifier

## Choose a process to execute
Let's choose which process we'll be executing.

In [ ]:
proc = wps.processes('CDAT.subset')[0]

## Configure our process with inputs and some constraints
Here we define our input variables and the domain we're interested in.

In [ ]:
tas = cwt.Variable('http://aims3.llnl.gov/thredds/dodsC/cmip5_css02_data/cmip5/output1/CMCC/CMCC-CM/decadal2005/mon/atmos/Amon/r1i2p1/tas/1/tas_Amon_CMCC-CM_decadal2005_r1i2p1_200511-201512.nc', 'tas')

d0 = cwt.Domain(lat=(-90, 0), time=(0, 2000))

## Let's execute the process
Here we'll execute the process on the server. We're using the inputs and domain we defined above and applying regridding to grid T21.

In [ ]:
wps.execute(proc, [tas], d0)

## We'll wait for the process to complete

In [ ]:
proc.wait()

## Let's checkout the output of the process

In [ ]:
print proc.output

## Let's explore the output

In [ ]:
import cdms2

f = cdms2.open(proc.output.uri)

data = f[proc.output.var_name]

# (66, 120, 480)
print data.shape
print data.getTime()[:]

In [ ]:
import vcs

v = vcs.init()

v.plot(data)